# Data Preprocessing

Here we create relevant Pandas DataFrames and save files as csv for the next steps of exploration and modelling. In particular, we will extract user ids and game ids to create a user-item interactions DataFrame, with each row being a particular user-item relationship.

In [1]:
# Import libraries
import pandas as pd
import numpy as np

#Import Warnings
import warnings
warnings.filterwarnings("ignore")

## Games Data

We will first load the gamesdata file, which has a row for each game and various descriptive features as columns. We notice there are features that are lists.



In [2]:
# Load games data
df = pd.read_json('gamesdata.json')
df.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140.0,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980.0,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290.0,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400.0,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570.0,NaN,NaN,NaN


In [3]:
# Save as csv
df.to_csv('gamesdata.csv')

## User items data

We now load the user items data, which has users as rows and details regarding items owned as columns.


In [4]:
# Load users/items data
useritems = pd.read_json('data.json')
useritems.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982480,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864384,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712560,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445856,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099488,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."



Whilst the items_count feature will not be relevant for the user-item interactions/ recommendation engine, it will be relevant to our stakeholders for the purpose of understanding their user base. As such let us extract it and save it for the data exploration phase.

In [5]:
# Extract items_count feat
numgames = useritems[['user_id', 'items_count']]
numgames.head()

,user_id,items_count
0,76561197970982479,277
1,js41637,888
2,evcentric,137
3,Riot-Punch,328
4,doctr,541


In [6]:
# Save as csv
numgames.to_csv('numgames.csv')

We note that the items column appears to be a list of dictionaries. Let us look at it in further detail.

In [7]:
# Preview items column values for first user
# Restrict to first 2 items in dictionary

useritems['items'][0][0:2]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0}]

Each game is represented by a dictionary with keys the game's item_id, item_name, playtime_forever and playtime_2weeks. The dictionaries are then storred in a list.

We will look to extract the item_ids into a seperate column. For now we will leave the playtime data but look to incorporate it later.

In [8]:
# Get all item_id for first user
gameids = [useritems['items'][0][index]['item_id'] for index, _ in enumerate(useritems['items'][0])]
# Show first 10 item ids
gameids[:10]

['10', '20', '30', '40', '50', '60', '70', '130', '300', '240']

We will now generalize the above and create a column extracting the item_id from the dictionaries for each user.

In [9]:
# Create column with item ids
useritems['item_id'] = useritems['items'].apply(lambda x: [x [index]['item_id'] for index, _ in enumerate(x)])
useritems.head()

,user_id,items_count,steam_id,user_url,items,item_id
0,76561197970982479,277,76561197970982480,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38..."
1,js41637,888,76561198035864384,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ..."
2,evcentric,137,76561198007712560,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3..."
3,Riot-Punch,328,76561197963445856,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300..."
4,doctr,541,76561198002099488,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...","[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100..."


As the unique user steam_id is a large integer, we will replace it with a new uid counter, which starts at 0 and increments by 1 (like the index).

We will also only select the relevant columns for the purpose of building a user-item interactions matrix, namely the newly created user id iud and the item_id.

In [10]:

# Add a column with substitute user_id, counter
useritems['uid'] = np.arange(len(useritems))

# Take relevant columns
useritems = useritems[['uid', 'item_id']]

# Check
useritems.head()

,uid,item_id
0,0,"[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38..."
1,1,"[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ..."
2,2,"[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3..."
3,3,"[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300..."
4,4,"[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100..."


The next step is to explode the item_id into seperate rows, so each user-item interaction has it's own row.



In [11]:
# Explode item_ids into seperate rows
lst_col = 'item_id'
useritems = pd.DataFrame({col:np.repeat(useritems[col].values, useritems[lst_col].str.len())
                              for col in useritems.columns.difference([lst_col])
                        }).assign(**{lst_col:np.concatenate(useritems[lst_col].values)})[useritems.columns.tolist()]
useritems

,uid,item_id
0,0,10
1,0,20
2,0,30
3,0,40
4,0,50
...,...,...
487305,4999,360730
487306,4999,460120
487307,4999,438490
487308,4999,460870


As we are concerned with whether the game is owned, as opposed to ratings, we will add a binary column owned which will have 1s everywhere, as only items owned appear in the DataFrame.

In [12]:
# Add binary owned column
useritems['owned'] = np.ones(shape = useritems.shape[0])

# Check
useritems.head()

,uid,item_id,owned
0,0,10,1.0
1,0,20,1.0
2,0,30,1.0
3,0,40,1.0
4,0,50,1.0


In [13]:
len(useritems)


487310

We note that we have over 487310 individual user-item relationships represented in our DataFrame.

We want to restrict ourselves to user-item relationships where the item is in the first gamesdata DataFrame to be able to extract relevant information such as genre.

We will ensure that the DataFrames can be merged on the game id feature by changing the type and column name.

In [14]:
# Change item_id to int
useritems['item_id'] = useritems['item_id'].astype(int)

# Rename column to match
useritems = useritems.rename(columns={'item_id': 'id'})

We can now merge the DataFrames.



In [15]:
# Merge useritems and games data dataframes
alldata = pd.merge(useritems, df, on = 'id')
alldata.head()

,uid,id,owned,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,developer,sentiment,metascore
0,0,10,1.0,Valve,[Action],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve,Overwhelmingly Positive,88
1,1,10,1.0,Valve,[Action],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve,Overwhelmingly Positive,88
2,3,10,1.0,Valve,[Action],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve,Overwhelmingly Positive,88
3,4,10,1.0,Valve,[Action],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve,Overwhelmingly Positive,88
4,10,10,1.0,Valve,[Action],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve,Overwhelmingly Positive,88


In [16]:
len(alldata)


408231

There are still 408231 individual user-item relationships recorded!



In [17]:
# Drop entries with no title
datawithnames = alldata.dropna(axis=0, subset=['title'])
datawithnames.head()

,uid,id,owned,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,developer,sentiment,metascore
0,0,10,1.0,Valve,[Action],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve,Overwhelmingly Positive,88
1,1,10,1.0,Valve,[Action],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve,Overwhelmingly Positive,88
2,3,10,1.0,Valve,[Action],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve,Overwhelmingly Positive,88
3,4,10,1.0,Valve,[Action],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve,Overwhelmingly Positive,88
4,10,10,1.0,Valve,[Action],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve,Overwhelmingly Positive,88


In [18]:
len(datawithnames)


400730

We will save this DataFrame as a csv file to conduct data exploration and gain insights.



In [19]:
# Save to csv
datawithnames.to_csv('mergeddata.csv')

Finally, let us extract the relevant columns for our user-item interactions matrix.



In [20]:
# Get relevant columns for recommendation engine
recdata = datawithnames[['uid','id','owned']]
recdata.head()

,uid,id,owned
0,0,10,1.0
1,1,10,1.0
2,3,10,1.0
3,4,10,1.0
4,10,10,1.0


In [21]:
# Save to csv
recdata.to_csv('recdata.csv')